<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [5]:
import pandas as pd
import warnings
import sklearn
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import optuna

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_percentage_error

from tqdm import notebook

In [6]:
factory_2019 = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2019 почасовки.csv', sep=';')
factory_2020 = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2020 почасовки.csv', sep=';')
factory_2021 = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2021 почасовки.csv', sep=';')
factory_2022 = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Предприятие_2//csv файлы/2022 почасовки.csv', sep=';')
factory = factory_2019.append(factory_2020).append(factory_2021).append(factory_2022).reset_index(drop=True)
weather = pd.read_csv('C://Users//Admin//Desktop//123//Самарэнерго//Погодные_данные/weather_new.csv', sep=';', 
                      on_bad_lines='skip', decimal=",")

In [ ]:
factory['datetime'] = factory['1']+' '+factory['2']
factory['datetime'] = pd.to_datetime(factory['datetime'], format='%d.%m.%Y %H:%M:%S')
factory = factory.rename(columns = {'3':'expenditure'})
factory = factory[['datetime', 'expenditure']]
factory = factory.set_index('datetime')

In [ ]:
weather = weather[['Местное время в Самаре','T']] #, 'U', 'Ff', 'N'
weather = weather.rename(columns = {'Местное время в Самаре':'datetime'})
weather['datetime'] = pd.to_datetime(weather['datetime'], format='%d.%m.%Y %H:%M')
weather = weather.set_index('datetime')
#weather['N'] = weather['N'].apply(cloud).apply(delited_1).apply(fill).astype('int')
weather = weather.fillna(method='bfill')
weather = weather.resample('1H').median()
weather = weather.reset_index()
for i in range(len(weather)-2):
    if weather['T'][i]==0:
        continue
    elif weather['T'][i]==len(weather)-1:
        break
    elif weather['T'][i]!='NaN':
        continue
    elif weather['T'][i]=='NaN' and weather['T'][i+2]=='NaN':
        weather['T'][i]= weather['T'][i-1]
    elif weather['T'][i]=='NaN' and weather['T'][i+1] =='NaN':
        weather['T'][i]= (weather['T'][i-1]+weather['T'][i+2])/2
    elif weather['T'][i]=='NaN' and weather['T'][i+1]!='NaN':
        weather['T'][i]= (weather['T'][i-1] +weather['T'][i+1])/2
weather = weather.set_index('datetime')
weather = weather.fillna(method='bfill')


from scipy import stats
from scipy.stats import kurtosis
from scipy.signal import find_peaks
data = factory.join(weather, on='datetime')
def make_features(data, max_lag, rolling_mean_size):
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    #data['is_weekend'] = data.dayofweek.isin([5,6])*1
    data['hour'] = data.index.hour
    data['expanding_window'] = data['expenditure'].expanding(3).mean()
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['expenditure'].shift(lag)
    data['rolling_mean'] = data['expenditure'].shift().rolling(rolling_mean_size).mean()
    #data['kurtosis'] = data['rolling_mean'].shift().rolling(3).apply(lambda x: kurtosis(x))
    #data['peaks'] = data['rolling_mean'].shift().rolling(3).apply(lambda x: len(find_peaks(x)[0]))
    
make_features(data, 24, 24)
data=data.dropna()
features = data.drop(['expenditure'], axis=1)
target = data['expenditure']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, shuffle=False)
model_lr = LinearRegression()
model_lr.fit(features, target)

In [104]:
try:
    #data_new = factory
    factory_new = factory[:'2022-06-24']
    data_new = weather.join(factory_new, on='datetime')
    make_features(data_new, 24, 24)
    data_new.dropna()
    print()
    print('Введите, с какого дня начать прогноз, в формате "ГГГГ-ММ-ДД":')
    first_date = input()
    print()
    print('Введите, по какой день (включительно) сделать прогноз в формате "ГГГГ-ММ-ДД":')
    secord_date = input()
    
    time_for_predict = data_new[str(first_date+' 00:00:00'):str(secord_date+' 23:00:00')].index.to_list()
    for i in notebook.tqdm(time_for_predict):
        target_a = data_new.expenditure[i]
        features_a = data_new[str(i):str(i)].drop(['expenditure'], axis=1)
        prediction = model_lr.predict(features_a)
        data_new.expenditure[str(i)] = (float(prediction))
        data_new = data_new[['T', 'expenditure']]
        make_features(data_new, 24, 24)
    prediction_april_test = data_new.expenditure[str(first_date+' 00:00:00'):]
    print('Предсказание завершено')
except:
    print()


Введите, с какого дня начать прогноз, в формате "ГГГГ-ММ-ДД":
2022-06-25

Введите, по какой день (включительно) сделать прогноз в формате "ГГГГ-ММ-ДД":
2022-06-27


  0%|          | 0/72 [00:00<?, ?it/s]

Предсказание завершено


In [105]:
print('Куда сохранить файл? Введите адрес в формате',  r'C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\1')
way_to_save = str(input())
prediction_april_test.to_csv(way_to_save) #и сохраняем
print()
print('Работа завершена. Данные сохранены в', way_to_save)

Куда сохранить файл? Введите адрес в формате C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\1
C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\1

Работа завершена. Данные сохранены в C:\Users\User\Desktop\123\Самарэнерго\Предприятие_2\Предсказания\Предсказания тесты\1
